In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import datetime
from os import listdir
import tensorflow as tf
import random
#from get_dataset import read_npy_dataset, split_npy_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from get_models import unet_dense, get_segment_model, get_Discriminator, get_GAN, get_Generator, save_model

/home/dudongsu/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dudongsu/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dudongsu/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dudongsu/anaconda3/envs/tf_gpu/lib

In [2]:
epochs = 2000
batch_size = 16
test_size = 0.1

In [3]:
# Dice Coefficient Loss Function
def dice_coefficient(y_true, y_pred):
    smoothing_factor = 1
    flat_y_true = y_true.flatten()
    flat_y_pred = y_pred.flatten()
    return (2. * np.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (np.sum(flat_y_true) + np.sum(flat_y_pred) + smoothing_factor)

In [4]:
def train_seg_model(model, splitted_npy_dataset_path, test_path, epochs):
   # test_XY = np.load(test_path+'/test.npy')
   # X_test, Y_test = test_XY[:,:,:,:,0:2],test_XY[:,:,:,:,2:3]

    batch_dirs = listdir(splitted_npy_dataset_path)
    len_batch_dirs = len(batch_dirs)

    if not os.path.exists('Data_v2/Checkpoints_UnetDense/'):
        os.makedirs('Data_v2/Checkpoints_UnetDense/')
    checkpoints = []
    checkpoints.append(EarlyStopping(monitor='val_loss', patience=500))
    checkpoints.append(ModelCheckpoint('Data_v2/Checkpoints_UnetDense/best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False))
  #  checkpoints.append(TensorBoard(log_dir='Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))
    model.fit_generator(data_gen(splitted_npy_dataset_path, batch_size), steps_per_epoch=int(len_batch_dirs/batch_size)+1, epochs=5000, validation_data=data_gen(test_path, 16), 
                          validation_steps=5, callbacks=checkpoints)
    #scores = model.evaluate(X_test, Y_test)        
    #print(Y_predict.shape)
    #dice_score = dice_coefficient(model.predict(X_test), Y_test)
        
  #  print('Test loss:', scores[0], '\nTest accuracy:', scores[1], '\nDice Coefficient Accuracy:', dice_score)
    return model

In [7]:
def train_gan(Generator, Encoder, Discriminator, GAN, splitted_npy_dataset_path, test_path, epochs):
    # Dataset sample sharing:。
    start_time = datetime.datetime.now()
    batch_dirs = listdir(splitted_npy_dataset_path)
    batch_num = int(len(batch_dirs)/batch_size)

    X_test = np.load(test_path+'/structures.npy')
    Y_test = np.load(test_path+'/dose.npy')

    data_generator = data_gen(splitted_npy_dataset_path, batch_size)
    fake = np.ones((batch_size,))
    valid = np.zeros((batch_size,))
    
    for epoch in range(epochs):
        print('Epoch: {0}/{1}'.format(epoch+1, epochs))

        print('Discriminator Training:')
        # Discriminator Training:
        for i in range(int(batch_num)):
            # Getting generated data:
            
            X_batch, Y_batch = next(data_generator)
            fake_dose = Generator.predict(X_batch)

            # Getting real data:
          #  getted_real_data = next(data_generator)

            # Getting batch:
            # Xs:
         #   dis_batch_X1 = np.concatenate((X_batch, getted_real_data[0]), axis=-1)
         #   dis_batch_X2 = np.concatenate((generator_pred, getted_real_data[1]), axis=0)
            # Ys:
         #   dis_batch_Y = np.concatenate((np.zeros((getted_generator_data[0].shape[0], 1)), np.ones((getted_real_data[0].shape[0], 1))), axis=0)

            Discriminator.trainable = True
            d_loss_real = Discriminator.train_on_batch([X_batch,Y_batch],valid) 
            d_loss_fake = Discriminator.train_on_batch([X_batch,fake_dose],fake)
            d_loss = 0.5*np.add(d_loss_real,d_loss_fake)
           # Discriminator.fit([dis_batch_X1, dis_batch_X2], dis_batch_Y, batch_size=1, epochs=1, shuffle=True)

     #   print('Generator Training:')
        # Generator Training:
     #   for i in range(int(batch_num/2)):
            # Getting generated data:
           # getted_generator_data = next(data_generator)

            # Getting batch:
            # Xs:
           # gan_batch_X = getted_generator_data[0]
            # Ys:
           # gan_batch_Y = np.ones((gan_batch_X.shape[0], 1))
      #  for i in range(int(batch_num*2)):
            Discriminator.trainable = False
        #    Encoder.trainable = True
           # GAN.fit(gan_batch_X, gan_batch_Y, batch_size=1, epochs=1, shuffle=True)
            g_loss = GAN.train_on_batch([X_batch], [valid, Y_batch])
            elapsed_time = datetime.datetime.now() - start_time
            
        scores = Generator.evaluate(X_test, Y_test)
        print('\nTest Loss:', scores[0])
        save_model(Generator, path='Data_v3/Checkpoints/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v3/Checkpoints/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')

        print('Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"')

    return Generator, Encoder, Discriminator

In [8]:
def data_gen(splitted_npy_dataset_path, batch_size):
    c = 0
    batch_dirs = listdir(splitted_npy_dataset_path)
    n = os.listdir(splitted_npy_dataset_path) #List of training images
    random.shuffle(n)
    while True:
        X_batch = np.zeros((batch_size, 128, 256, 13)).astype('float')
        Y_batch = np.zeros((batch_size, 128, 256, 1)).astype('float')
        for i in range(c,c+batch_size):
            batch_XY = np.load(splitted_npy_dataset_path+'/'+n[i])
            #print('batch_XY size', batch_XY.shape)
            X_batch[i-c] =  batch_XY[:,:,:,0:13]
            Y_batch[i-c] =  batch_XY[:,:,:,13:14]    
        c = c + batch_size
        if(c+batch_size>=len(os.listdir(splitted_npy_dataset_path))):
            c=0;
            random.shuffle(n)
        yield X_batch, Y_batch

In [9]:
def main(train_gan_model = True, ):
    if train_gan_model:
        Generator = unet_dense(input_size = (128, 256, 13))
        Encoder = unet_dense(input_size = (128, 256, 13))
        Discriminator = get_Discriminator(input_shape_1=(128, 256, 13), input_shape_2=(128, 256, 1)) #, Encoder = Encoder)
        GAN = get_GAN((128, 256, 13), Generator, Discriminator)

        # Saving non-trained models:
        save_model(Generator, path='Data_v3/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Encoder, path='Data_v3/GAN-Models/Encoder/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v3/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')
        print('Non-Trained model saved to "Data/GAN-Models"!')
        ## Train
        Generator, Encoder, Discriminator = train_gan(Generator, Encoder, Discriminator, GAN, splitted_npy_dataset_path='Data_v3/npy_dataset/training_npy_dataset', test_path = 'Data_v3/npy_dataset/validation_npy', epochs = epochs)

        # Saving trained models:
        save_model(Generator, path='Data_v3/GAN-Models/Generator/', model_name = 'model', weights_name = 'weights')
        save_model(Encoder, path='Data_v3/GAN-Models/Encoder/', model_name = 'model', weights_name = 'weights')
        save_model(Discriminator, path='Data_v3/GAN-Models/Discriminator/', model_name = 'model', weights_name = 'weights')
        print('Trained model saved to "Data/GAN-Models"!')
        return Generator
        
    else:
        segment_model = unet_dense(input_size = (128, 256, 13))
        save_model(segment_model, path='Data_v3/Model_UnetDense/', model_name = 'model', weights_name = 'weights')
        print('Non-Trained model saved to "Data_v3/Model_UnetDense"!')
        model = train_seg_model(segment_model, splitted_npy_dataset_path='Data_v3/npy_dataset/training_npy_dataset', test_path = 'Data_v3/npy_dataset/validation_npy', epochs = epochs)
        save_model(model, path='Data_v3/Model_UnetDense/', model_name = 'model', weights_name = 'weights')
        print('Trained model saved to "Data_V2/Model_UnetDense"!')
        return segment_model

In [11]:
if __name__ == '__main__':
    main(train_gan_model = True)

output shape= (?, 128, 256, 1)
Dose prediction Model Architecture:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 128, 256, 13) 0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 128, 256, 16) 1888        input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_45 (Concatenate)    (None, 128, 256, 29) 0           input_6[0][0]                    
                                                                 conv2d_53[0][0]                  
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 128

output shape= (?, 128, 256, 1)
Dose prediction Model Architecture:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 128, 256, 13) 0                                            
__________________________________________________________________________________________________
conv2d_77 (Conv2D)              (None, 128, 256, 16) 1888        input_7[0][0]                    
__________________________________________________________________________________________________
concatenate_67 (Concatenate)    (None, 128, 256, 29) 0           input_7[0][0]                    
                                                                 conv2d_77[0][0]                  
__________________________________________________________________________________________________
conv2d_78 (Conv2D)              (None, 128

Discriminator Architecture:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 128, 256, 13) 0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 128, 256, 1)  0                                            
__________________________________________________________________________________________________
concatenate_89 (Concatenate)    (None, 128, 256, 14) 0           input_8[0][0]                    
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_101 (Conv2D)             (None, 64, 128, 32)  7200        concatenate_89[0

55/55 [==============================] - 0s 9ms/step

Test Loss: 232.5607596657493
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 10/2000
Discriminator Training:
55/55 [==============================] - 0s 9ms/step

Test Loss: 204.17409002130682
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 11/2000
Discriminator Training:
55/55 [==============================] - 0s 9ms/step

Test Loss: 197.0123185591264
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 12/2000
Discriminator Training:
55/55 [==============================] - 0s 9ms/step

Test Loss: 244.9063826127486
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 13/2000
Discriminator Training:
55/55 [==============================] - 0s 9ms/step

Test Loss: 168.6175057151101
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1

55/55 [==============================] - 0s 8ms/step

Test Loss: 157.74058172052557
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 50/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 167.16124378551137
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 51/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 174.294077092951
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 52/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 160.5207050670277
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 53/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 194.7619074041193
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 5

...

Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1839/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 43.62088109796697
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1840/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 51.67785672274503
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1841/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 45.94756109064276
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1842/2000
Discriminator Training:
55/55 [==============================] - 0s 7ms/step

Test Loss: 40.45852369828658
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1843/2000
Discriminator Training:
55/55 [==============================] - 0s

55/55 [==============================] - 0s 8ms/step

Test Loss: 47.51088756214489
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1879/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 43.308901283957745
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1880/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 44.691827531294386
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1881/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 43.209504560990766
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1882/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 46.38521367853338
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/

Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1918/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 48.62880609685725
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1919/2000
Discriminator Training:
55/55 [==============================] - 0s 7ms/step

Test Loss: 43.66806071888317
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1920/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 44.45599725896662
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1921/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 50.261305930397725
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1922/2000
Discriminator Training:
55/55 [==============================] - 0

55/55 [==============================] - 0s 8ms/step

Test Loss: 44.8945971402255
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1958/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 44.53012945001775
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1959/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 46.68057556152344
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1960/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 45.240366571599786
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1961/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 46.12988419966264
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
E

Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1997/2000
Discriminator Training:
55/55 [==============================] - 0s 7ms/step

Test Loss: 45.923456642844464
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1998/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 45.30429590398615
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 1999/2000
Discriminator Training:
55/55 [==============================] - 0s 8ms/step

Test Loss: 44.84738880504261
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Epoch: 2000/2000
Discriminator Training:
55/55 [==============================] - 0s 7ms/step

Test Loss: 45.41808929443359
Segmentation model checkpoints saved to "Data_v2/Chackpoints/GAN-Models/Generator/"
Trained model saved to "Data/GAN-Models"!


In [19]:
# Training Segment Model:
def train_prediction_model(model, splitted_npy_dataset_path, test_path, epochs):
    
    batch_dirs = listdir(splitted_npy_dataset_path)
    len_batch_dirs = len(batch_dirs)

    if not os.path.exists('Data/Checkpoints/'):
        os.makedirs('Data/Checkpoints/')
    checkpoints = []
    checkpoints.append(ModelCheckpoint('Data/Checkpoints/best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True))
  #  checkpoints.append(TensorBoard(log_dir='Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))
    model.fit_generator(data_gen(splitted_npy_dataset_path, batch_size), steps_per_epoch=int(len_batch_dirs/batch_size)+1, epochs=100, validation_data=data_gen(test_path, 5), 
                          validation_steps=1, callbacks=checkpoints)
    scores = model.evaluate(X_test, Y_test)        
    print(Y_predict.shape)
    dice_score = dice_coefficient(model.predict(X_test), Y_test)
        
  #  print('Test loss:', scores[0], '\nTest accuracy:', scores[1], '\nDice Coefficient Accuracy:', dice_score)
    return model